In [1]:
import os
from pathlib import Path
from pprint import pprint
import re
import shutil

import numpy as np
import pandas as pd
from pandas import DataFrame

%cd /root/nerve/python
from nerve.parser import AssetNameParser

/root/nerve/python


In [2]:
def list_all_files(directory):
    if not isinstance(directory, Path):
        directory = Path(directory)

    if not directory.is_dir():
        msg = f'{directory} is not a directory or does not exist.'
        raise FileNotFoundError(msg)

    for root, dirs, files in os.walk(directory):
        for file in files:
            yield Path(root, file)
            
def directory_to_dataframe(directory, include_pattern='', exclude_pattern='\.DS_Store'):
    files = list_all_files(directory)
    files = filter(lambda x: re.search(include_pattern, x.absolute().as_posix()), files)
    files = filter(lambda x: not re.search(exclude_pattern, x.absolute().as_posix()), files)
    files = sorted(list(files))
    
    data = DataFrame()
    data['fullpath'] = files
    data['filename'] = data.fullpath.apply(lambda x: x.name)
    data['extension'] = data.fullpath.apply(lambda x: os.path.splitext(x)[-1][1:])
    data.fullpath = data.fullpath.apply(lambda x: x.absolute().as_posix())
    return data

def try_(function, item, return_item='item'):
    try:
        return function(item)
    except Exception as error:
        if return_item == 'item':
            return item
        elif return_item == 'error':
            return error
        return return_item

In [3]:
root = '/tmp'
src = Path(root, 'proj001')
if src.exists():
    shutil.rmtree(src)

fields = [
    'project',
    'specification',
    'descriptor',
    'version',
    'coordinate',
    'frame',
    'extension'
]
parser = AssetNameParser(fields)
rev_parser = AssetNameParser(
    list(reversed(fields[:-1])) + fields[-1:]
)
dirparser = AssetNameParser(fields[:-2])

ys = range(0, 3)
frames = range(1, 11)
for y in ys:
    for i in frames:
        m = dict(
            project='proj001',
            specification='spec002',
            descriptor='desc',
            version=3,
            coordinate=[0, y],
            frame=i,
            extension='exr',
        )

        filename = parser.to_string(m)
        dirname = dirparser.to_string(m)

        if i == 5:
            filename = re.sub('d-.*?_', 'd-DESC_', filename)
            
        if i == 7:
            filename = rev_parser.to_string(m)
            
        if i == 9:
            filename = 'bad-filename.txt'

        fullpath = Path(
            src,
            m['specification'],
            m['descriptor'],
            dirname,
            filename
        )
        os.makedirs(fullpath.parent, exist_ok=True)
        with open(fullpath, 'w') as f:
            f.write('')

In [21]:
class LocalDataBase:
    def __init__(
        self,
        root,
        specifications,
        include_pattern='',
        exclude_pattern=r'\.DS_Store',
        ignore_order=False
    ):
        if not isinstance(root, Path):
            root = Path(root)
        if not root.is_dir():
            msg = f'{root} is not a directory or does not exist.'
            raise FileNotFoundError(msg)

        self.root = root
        self.include_pattern = include_pattern
        self.exclude_pattern = exclude_pattern
        self.ignore_order = ignore_order
        self.data = None
        self._specifications = {x.name: x for x in specifications}

    def update(self):
        # get file data
        data = directory_to_dataframe(
            self.root,
            include_pattern=self.include_pattern,
            exclude_pattern=self.exclude_pattern
        )

        # get specification data
        spec = data.filename.apply(
            lambda x: try_(
                AssetNameParser.parse_specification,
                x,
                'error'
            )
        )
        
        # convert spec parse errors to dict
        mask = spec.apply(lambda x: not isinstance(x, dict)).astype(bool)
        spec[mask] = spec[mask].apply(lambda x: dict(error=x))

        # flatten spec data into DataFrame        
        spec = DataFrame(spec.tolist())
        if 'error' not in spec.columns:
            spec['error'] = None
        
        # combine spec data with file data
        data = pd.concat([data, spec], axis=1)
        
        # get mask of files with specs        
        mask = data.specification.apply(
            lambda x: isinstance(x, str) and x in self._specifications.keys()
        )
        mask = data[mask].index

        # create meta column
        data['meta'] = None
        data.meta = data.meta.apply(lambda x: {})
        
        # parse filenames for metadata         
        parse = lambda x: AssetNameParser(
            self._specifications[x[0]].fields
        ).parse(x[1], ignore_order=self.ignore_order)
        data.loc[mask, 'meta'] = data.loc[mask]\
            .apply(lambda x: [x.specification, x.filename], axis=1)\
            .apply(lambda x: try_(parse, x, 'error'))\
            .apply(lambda x: x if isinstance(x, dict) else dict(error=x))
        
        # concatenate data and metadata         
        meta = data.meta.tolist()
        meta = DataFrame(meta)

        cols = ['specification', 'extension', 'error']
        for col in cols:
            if col not in meta.columns:
                meta[col] = None

        meta['err'] = meta.error
        meta.drop(columns=cols, inplace=True)
        data = pd.concat([data, meta], axis=1)
        data.loc[mask, 'error'] = data.loc[mask, 'err']
        
        # cleanup columns         
        cols = [
            'project',
            'specification',
            'descriptor',
            'version',
            'coordinate',
            'frame',
            'extension',
            'filename',
            'fullpath',
            'error'
        ]
        for col in cols:
            if col not in data.columns:
                data[col] = None
        data = data[cols]

        self.data = data
        return self
    
class Spec001:
    name = 'spec001'
    fields = [
        'project',
        'specification',
        'descriptor',
        'version',
        'coordinate',
        'frame',
        'extension'
    ]
    
class Spec002:
    name = 'spec002'
    fields = [
        'project',
        'specification',
        'descriptor',
        'version',
        'coordinate',
        'frame',
        'extension'
    ]

db = LocalDataBase(src, [Spec002])
# db = LocalDataBase(src, [Spec002], ignore_order=True)
data = db.update().data
data

,project,specification,descriptor,version,coordinate,frame,extension,filename,fullpath,error
0,NaN,NaN,NaN,NaN,NaN,NaN,txt,bad-filename.txt,/tmp/proj001/spec002/desc/p-proj001_s-spec002_...,"Specification not found in ""bad-filename.txt""...."
1,NaN,spec002,NaN,NaN,NaN,NaN,exr,f0007_c000-000_v003_d-desc_s-spec002_p-proj001...,/tmp/proj001/spec002/desc/p-proj001_s-spec002_...,"Incorrect field order in ""f0007_c000-000_v003_..."
2,NaN,spec002,NaN,NaN,NaN,NaN,exr,p-proj001_s-spec002_d-DESC_v003_c000-000_f0005...,/tmp/proj001/spec002/desc/p-proj001_s-spec002_...,"Illegal descriptor field token in ""p-proj001_s..."
3,proj001,spec002,desc,3.0,"[0, 0]",1.0,exr,p-proj001_s-spec002_d-desc_v003_c000-000_f0001...,/tmp/proj001/spec002/desc/p-proj001_s-spec002_...,NaN
4,proj001,spec002,desc,3.0,"[0, 0]",2.0,exr,p-proj001_s-spec002_d-desc_v003_c000-000_f0002...,/tmp/proj001/spec002/desc/p-proj001_s-spec002_...,NaN
5,proj001,spec002,desc,3.0,"[0, 0]",3.0,exr,p-proj001_s-spec002_d-desc_v003_c000-000_f0003...,/tmp/proj001/spec002/desc/p-proj001_s-spec002_...,NaN
6,proj001,spec002,desc,3.0,"[0, 0]",4.0,exr,p-proj001_s-spec002_d-desc_v003_c000-000_f0004...,/tmp/proj001/spec002/desc/p-proj001_s-spec002_...,NaN
7,proj001,spec002,desc,3.0,"[0, 0]",6.0,exr,p-proj001_s-spec002_d-desc_v003_c000-000_f0006...,/tmp/proj001/spec002/desc/p-proj001_s-spec002_...,NaN
8,proj001,spec002,desc,3.0,"[0, 0]",8.0,exr,p-proj001_s-spec002_d-desc_v003_c000-000_f0008...,/tmp/proj001/spec002/desc/p-proj001_s-spec002_...,NaN
9,proj001,spec002,desc,3.0,"[0, 0]",10.0,exr,p-proj001_s-spec002_d-desc_v003_c000-000_f0010...,/tmp/proj001/spec002/desc/p-proj001_s-spec002_...,NaN


In [7]:
list(map(print, data[data.error.notnull()].error.tolist()))

Specification not found in "bad-filename.txt". (at char 0), (line:1, col:1)
Incorrect field order in "f0007_c000-000_v003_d-desc_s-spec002_p-proj001.exr". Given field order: ['project', 'specification', 'descriptor', 'version', 'coordinate', 'frame', 'extension']. (at char 0), (line:1, col:1)
Illegal descriptor field token in "p-proj001_s-spec002_d-DESC_v003_c000-000_f0005.exr". Expecting: [a-z0-9][a-z0-9-]* (at char 0), (line:1, col:1) (at char 0), (line:1, col:1)
Specification not found in "bad-filename.txt". (at char 0), (line:1, col:1)
Incorrect field order in "f0007_c000-001_v003_d-desc_s-spec002_p-proj001.exr". Given field order: ['project', 'specification', 'descriptor', 'version', 'coordinate', 'frame', 'extension']. (at char 0), (line:1, col:1)
Illegal descriptor field token in "p-proj001_s-spec002_d-DESC_v003_c000-001_f0005.exr". Expecting: [a-z0-9][a-z0-9-]* (at char 0), (line:1, col:1) (at char 0), (line:1, col:1)
Specification not found in "bad-filename.txt". (at char 0), 

[None, None, None, None, None, None, None, None, None]

In [9]:
!tree /tmp/proj001

/tmp/proj001
└── spec002
    └── desc
        ├── p-proj001_s-spec002_d-desc_v003_c000-000
        │   ├── bad-filename.txt
        │   ├── f0007_c000-000_v003_d-desc_s-spec002_p-proj001.exr
        │   ├── p-proj001_s-spec002_d-DESC_v003_c000-000_f0005.exr
        │   ├── p-proj001_s-spec002_d-desc_v003_c000-000_f0001.exr
        │   ├── p-proj001_s-spec002_d-desc_v003_c000-000_f0002.exr
        │   ├── p-proj001_s-spec002_d-desc_v003_c000-000_f0003.exr
        │   ├── p-proj001_s-spec002_d-desc_v003_c000-000_f0004.exr
        │   ├── p-proj001_s-spec002_d-desc_v003_c000-000_f0006.exr
        │   ├── p-proj001_s-spec002_d-desc_v003_c000-000_f0008.exr
        │   └── p-proj001_s-spec002_d-desc_v003_c000-000_f0010.exr
        ├── p-proj001_s-spec002_d-desc_v003_c000-001
        │   ├── bad-filename.txt
        │   ├── f0007_c000-001_v003_d-desc_s-spec002_p-proj001.exr
        │   ├── p-proj001_s-spec002_d-DESC_v003_c000-001_f0005.exr
        │   ├── p-proj001_s-spec002_d-desc_v003_c000-

In [ ]:
import re
import shutil
import pandas as pd
from pandas import DataFrame, Series
import github3
from github3 import login
from git import Repo, GitCommandError
import yaml
from subprocess import Popen, PIPE, STDOUT, SubprocessError, check_output
from configparser import ConfigParser
from github3.repos.branch import Branch
from github3.null import NullObject
from schematics.models import Model, BaseType
from schematics.types import StringType, IntType, BooleanType
from schematics.types.compound import ListType, DictType
from schematics.exceptions import ValidationError
%cd ~/Documents/projects/stitch/python/
from stitch.core.stitch_frame import StitchFrame
%cd ~/Documents/projects/nerve/python/
from nerve.core.project_manager import ProjectManager
from nerve.core.project import Project
from nerve.core.git_remote import GitRemote
from nerve.core.utils import *
from nerve.spec.base import *
from nerve.core.git import Git
from nerve.core.git_lfs import GitLFS
from nerve.core.metadata import Metadata
from nerve.core.utils import execute_subprocess, get_status
# from nerve.core.api import *
from nerve.spec.specifications import *
from nerve.spec.validators import *
from nerve.spec.traits import *
from nerve.spec import traits
from nerve.test.test_suite import *
from nerve.spec.test import *
%cd /tmp/ntr002

In [6]:
#! /usr/bin/env python
'''
'''
# ------------------------------------------------------------------------------

import os
import re
from functools import wraps
from schematics.exceptions import ValidationError
import nerve
from nerve.core.utils import Name
# ------------------------------------------------------------------------------

def validator(func):
    @wraps(func)
    def wrapper(_self):
        if not func(_self):
            message = 'is not a valid ' + _self.name
            raise ValidationError(str(_self.value) + ' ' + message)
        return True
    return wrapper

class Trait(object):
    def __init__(self, fullpath):
        self._fullpath = fullpath
        self._name_traits = Name(fullpath).to_dict()
        self._name = re.sub('Trait', '', self.__class__.__name__)

    @property
    def name(self):
        return self._name

    @property
    def name_traits(self):
        return self._name_traits

    @property
    def fullpath(self):
        return self._fullpath

    @property
    def value(self):
        raise NotImplementedError('Please define the value proprty of your subclass')

    def validate(self, message=None):
        return True
# ------------------------------------------------------------------------------

class SpecificationTrait(Trait):
    @property
    def value(self):
        return self.name_traits['specification']

    @validator
    def validate(self):
        return hasattr(nerve.spec.specifications, self.value.capitalize())
# ------------------------------------------------------------------------------

# def main():
#     '''
#     Run help if called directly
#     '''

#     import __main__
#     help(__main__)
# # ------------------------------------------------------------------------------

# # __all__ = []

# if __name__ == '__main__':
#     main()


s = SpecificationTrait('/tmp/ntr002/ntr002_proj001_meta.yml')
s.name_traits
# s.validate()

{'config': False,
 'coordinate': None,
 'descriptor': None,
 'extension': 'yml',
 'frame': None,
 'ftype': 'project',
 'fullpath': '/tmp/ntr002/ntr002_proj001_meta.yml',
 'meta': True,
 'project-name': 'ntr002',
 'project-path': '/tmp/ntr002',
 'project-root': '/tmp',
 'raw': '/tmp/ntr002/ntr002_proj001_meta.yml',
 'render-pass': None,
 'specification': 'proj001',
 'specification-path': None,
 'template': False,
 'version': None}

In [2]:
conf = '/Users/alex/Documents/projects/nerve/python/nerve/resources/nerverc.yml'
man = ProjectManager(conf)

In [3]:
man.delete('ntr002', True, True)

/Users/alex/Documents/projects/nerve/python/nerve/core/project_manager.py:101: UserWarning: /tmp/ntr002 project metadata does not exist
  warn(msg)
/Users/alex/Documents/projects/nerve/python/nerve/core/project_manager.py:473: UserWarning: /tmp/ntr002 is not a project directory.  Local deletion aborted.
  warn(info.path + ' is not a project directory.  Local deletion aborted.')


False

In [3]:
man.create('ntr002', 'some note about a thing')

/Users/alex/Documents/projects/nerve/python/nerve/core/project_manager.py:101: UserWarning: /tmp/ntr002 project metadata does not exist
  warn(msg)


True

In [3]:
man._config['timeout'] = 5
man.clone('ntr002')
man._config['timeout'] = 100

In [4]:
%%bash
rm -rf /tmp/ntr002-new
cp -R /tmp/ntr002/ /tmp/ntr002-old
cd /tmp/ntr002/
mkdir vol001/ntr002_vol001_desc_v001
echo v001 sdjfkh 0000 > vol001/ntr002_vol001_desc_v001/ntr002_vol001_desc_v001_0000.png
echo v001 jyelkh 0001 > vol001/ntr002_vol001_desc_v001/ntr002_vol001_desc_v001_0001.png
echo v001 lwekfj 0002 > vol001/ntr002_vol001_desc_v001/ntr002_vol001_desc_v001_0002.png
echo vasc v001 > maya001/ntr002_maya001_vasc_v001.mb
echo kidney v001 > maya001/ntr002_maya001_kdny_v001.mb
tree

.
├── geo001
├── maya001
│   ├── ntr002_maya001_kdny_v001.mb
│   └── ntr002_maya001_vasc_v001.mb
├── ntr002_proj001_meta.yml
└── vol001
    └── ntr002_vol001_desc_v001
        ├── ntr002_vol001_desc_v001_0000.png
        ├── ntr002_vol001_desc_v001_0001.png
        └── ntr002_vol001_desc_v001_0002.png

4 directories, 6 files


In [5]:
stat = list(man.status('ntr002'))
data = None
if len(stat) > 0:
    data = DataFrame(list(stat)).applymap(lambda x: x.data)
    data = StitchFrame(data).flatten(prefix=False).to_dataframe().T
data

,0,1,2
asset-id,76a1dab5-5740-4892-a63c-47...,2f008bd7-41e5-405d-8b48-a8...,6225373a-5b1f-4eb7-9ea1-d2...
asset-name,ntr002_maya001_kdny_v001,ntr002_maya001_vasc_v001,ntr002_vol001_desc_v001
asset-type,nondeliverable,nondeliverable,deliverable
data,None,None,None
dependencies,NaN,NaN,[]
descriptor,NaN,NaN,desc
notes,,,
project-id,74626471,74626471,74626471
project-name,ntr002,ntr002,ntr002
project-url,git@github.com:nerve-test-...,git@github.com:nerve-test-...,git@github.com:nerve-test-...


In [6]:
man.publish('ntr002', 'some note from the alex', verbosity=2)

True

In [7]:
%%bash
mv /tmp/ntr002 /tmp/ntr002-new
mv /tmp/ntr002-old/ /tmp/ntr002

In [8]:
man.request('ntr002')

True